In [52]:
### Ref: https://github.com/loliverhennigh/Convolutional-LSTM-in-Tensorflow

In [3]:
import time
import importlib
import numpy as np
import tensorflow as tf
from tensorflow.contrib.slim import fully_connected as fc
import matplotlib.pyplot as plt 
import glob
from collections import OrderedDict
%matplotlib inline
from datetime import datetime
import os
from pathlib import Path
import layer_def as ld
from BasicConvLSTMCell import BasicConvLSTMCell
importlib.reload(ld)
train_files = "/Users/gongbing/PycharmProjects/video_prediction_savp/data/era5_size_64_64_3_3t_norm/train"
test_files = "/Users/gongbing/PycharmProjects/video_prediction_savp/data/era5_size_64_64_3_3t_norm/test"
val_files = "/Users/gongbing/PycharmProjects/video_prediction_savp/data/era5_size_64_64_3_3t_norm/train"
#hyparameters
num_epochs = 50
batch_size=40
input_dim=3
num_sample = 1500

In [93]:
def make_dataset(type="train"):
    if type=="train": filenames = glob.glob("/Users/gongbing/PycharmProjects/video_prediction_savp/data/era5_size_64_64_3_3t_norm/train/*.tfrecords")
    if type=="val":filenames = glob.glob("/Users/gongbing/PycharmProjects/video_prediction_savp/data/era5_size_64_64_3_3t_norm/val/*.tfrecords")
    if type=="test":filenames = glob.glob("/Users/gongbing/PycharmProjects/video_prediction_savp/data/era5_size_64_64_3_3t_norm/test/*.tfrecords")
    
    def parser(serialized_example):
            seqs = OrderedDict()
            keys_to_features = {
                # 'width': tf.FixedLenFeature([], tf.int64),
                # 'height': tf.FixedLenFeature([], tf.int64),
                'sequence_length': tf.FixedLenFeature([], tf.int64),
                # 'channels': tf.FixedLenFeature([],tf.int64),
                # 'images/encoded':  tf.FixedLenFeature([], tf.string)
                'images/encoded': tf.VarLenFeature(tf.float32)
            }
            parsed_features = tf.parse_single_example(serialized_example, keys_to_features)
            seq = tf.sparse_tensor_to_dense(parsed_features["images/encoded"])
            images = tf.reshape(seq, [20,64, 64,3], name = "reshape_new")
            seqs["images"] = images
            return seqs
    dataset = tf.data.TFRecordDataset(filenames, buffer_size = 8 * 1024 * 1024)
    dataset = dataset.repeat(num_epochs)
    dataset = dataset.apply(tf.contrib.data.map_and_batch(
            parser, batch_size, drop_remainder = True, num_parallel_calls = None))
    
    dataset = dataset.prefetch(batch_size) 
    iterator = dataset.make_initializable_iterator()
    return iterator


class convLSTM(object):

    def __init__(self, learning_rate=1e-4, batch_size=64,n_z=0,model_name="convLSTM1",latent_loss_weight=0.0):
        # Set hyperparameters
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.n_z=0
        self.latent_loss_weight=latent_loss_weight
        self.model_name = model_name
        # Build the graph
        self.build()
        # Initialize paramters
        self.sess = tf.InteractiveSession()
        self.sess.run(tf.global_variables_initializer())
        # Summary op
        self.loss_summary = tf.summary.scalar("total_losses", self.total_loss)

        self.summary_op = tf.summary.merge_all()
        self.summary_dir = "./"
        self.base_dir = model_name
        self.checkpoint_dir = self.base_dir + "/checkpoint"
        Path(self.checkpoint_dir).mkdir(parents=True, exist_ok=True)
        self.train_log_file = self.base_dir + "/train_" + datetime.now().strftime("%Y%m%d-%H%M%S")
        self.val_log_file = self.base_dir + "/val_" + datetime.now().strftime("%Y%m%d-%H%M%S")
        self.train_writer = tf.summary.FileWriter(self.train_log_file, self.sess.graph)
        self.val_writer = tf.summary.FileWriter(self.val_log_file, self.sess.graph)
        
        
    def myModel(self,model_name="convLSTM1",*args):
        modelsDic = {
             "convLSTM1": self.convLSTM_network(*args),
               }
        self.model = modelsDic[model_name]
        return self.model
         
    @staticmethod
    def convLSTM_cell(inputs, hidden, nz=16):
        print("Inputs shape", inputs.shape)
        conv1 = ld.conv_layer(inputs, 3, 2, 8, "encode_1")
        print("Encode_1_shape",conv1.shape) 
        conv2 = ld.conv_layer(conv1, 3, 1, 8, "encode_2")
        print("Encode 2_shape,",conv2.shape)
        conv3 = ld.conv_layer(conv2, 3, 2, 8, "encode_3") 
        print("Encode 3_shape, ", conv3.shape)
        y_0 = conv3
        # conv lstm cell
        with tf.variable_scope('conv_lstm', initializer=tf.random_uniform_initializer(-.01, 0.1)):
            #cell = BasicConvLSTMCell(shape=[8, 8], filter_size=[3, 3],num_features=4)
            cell = BasicConvLSTMCell(shape = [16, 16], filter_size = [3, 3], num_features=8)
            if hidden is None:
                hidden = cell.zero_state(y_0, tf.float32)
                print ("hidden zero layer",hidden.shape)
            y_1, hidden = cell(y_0, hidden)

            
        y1_shape = y_1.get_shape().as_list()
        print ("y1_shape,",y1_shape)
        
        z3 = tf.reshape(y_1, [-1, y1_shape[1], y1_shape[2], y1_shape[3]])
     
        conv5 = ld.transpose_conv_layer(z3, 3, 2, 8, "decode_5") 
        print("conv5 shape",conv5)
       
        conv6 = ld.transpose_conv_layer(conv5, 3, 1, 8, "decode_6") 
        print("conv6 shape",conv6)

        x_hat = ld.transpose_conv_layer(conv6, 3, 2, 3, "decode_7")  # set activation to linear
        print("x hat shape",x_hat)
        return x_hat, hidden
    
    
    def convLSTM_network(self):
        network_template = tf.make_template('network', convLSTM.convLSTM_cell) #make the template to share the variables
        # create network
        x_unwrap = []
        seq_start = 1
        hidden = None
        for i in range(20):
            if i < seq_start:
                #x_1, hidden, y_pred = network_template(x_dropout[:, i, :, :, :], hidden)
                x_1, hidden = network_template(self.x[:, i, :, :, :], hidden)
            else:
                x_1, hidden = network_template(x_1, hidden)
            x_unwrap.append(x_1)

        # pack them all together
        x_hat = tf.stack(x_unwrap)
        self.x_hat = tf.transpose(x_hat, [1, 0, 2, 3, 4]) #change first dim with sec dim
        print ("X_HAT", self.x_hat)
        return self.x_hat
        
        
    # Build the netowrk and the loss functions
    def build(self):
        tf.reset_default_graph()
        self.train_iterator = make_dataset(type="train")
        self.val_iterator = make_dataset(type="val")
        self.test_iterator = make_dataset(type="test")
        self.x = tf.placeholder(tf.float32, [None,20,64,64,3])
        self.global_step = tf.Variable(0, name='global_step', trainable=False)
        self.increment_global_step = tf.assign_add(self.global_step,1,name = 'increment_global_step')
        
        #ARCHITECTURE
        self.x_hat = self.myModel(model_name=self.model_name)
        
        #Loss calculation
        self.total_loss = tf.reduce_mean(tf.square(self.x[:,1:,:,:,0]  - self.x_hat[:,:-1,:,:,0]))
        
        self.train_op = tf.train.AdamOptimizer(
            learning_rate=self.learning_rate).minimize(self.total_loss,global_step=self.global_step)
        
        # Build a saver
        self.saver = tf.train.Saver(tf.global_variables())
        
        return

    # Execute the forward and the backward pass
    def run_single_step(self):
        global_step = self.sess.run(self.global_step)
        try:
            train_batch = self.sess.run(self.train_iterator.get_next())
            print("Train_batch shape", train_batch["images"].shape)
            x_hat, train_summary, _, train_losses = self.sess.run([self.x_hat, self.summary_op, self.train_op, self.total_loss], feed_dict={self.x: train_batch["images"]})
            self.train_writer.add_summary(train_summary, global_step)
            print("x_hat.shape", x_hat.shape)
        except tf.errors.OutOfRangeError:
            print("train out of range error")
        
        try:
            val_batch = self.sess.run(self.val_iterator.get_next())
            val_summary, _, val_losses = self.sess.run([self.summary_op,self.train_op, self.total_loss], feed_dict={self.x: val_batch["images"]})
            self.val_writer.add_summary(val_summary, global_step)
        except tf.errors.OutOfRangeError:
            print("train out of range error")
        
        return train_losses,val_losses

    # x -> x_hat
    def reconstructor(self, x):
        x_hat = self.sess.run(self.x_hat, feed_dict={self.x: x})
        return x_hat

    # z -> x
    def generator(self, z):
        x_hat = self.sess.run(self.x_hat, feed_dict={self.z: z})
        return x_hat
    
    
    # x -> z
    def transformer(self, x):
        z = self.sess.run(self.z, feed_dict={self.x: x})
        return z


In [94]:
def trainer_and_checkpoint(model_class, learning_rate=1e-4, 
            batch_size=64, num_epoch=100, n_z=16, log_step=5):
    #restore the existing checkpoints  
    model = model_class(learning_rate=learning_rate, batch_size=batch_size, n_z=n_z)
    ckpt = tf.train.get_checkpoint_state(model.checkpoint_dir)
    if ckpt and ckpt.model_checkpoint_path:
        #Extract from checkpoint filename
        global_step = int(os.path.basename(ckpt.model_checkpoint_path).split('-')[1])
        sess = tf.Session()
        print("Restore from {}".format(ckpt.model_checkpoint_path))
        graph = tf.get_default_graph()
        saver = tf.train.Saver(tf.global_variables())
        #saver = tf.train.import_meta_graph(os.path.join(model.checkpoint_dir,'model.ckpt-{}.meta'.format(global_step)))
        saver.restore(sess,tf.train.latest_checkpoint(model.checkpoint_dir))
        loaded_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=tf.get_variable_scope().name)
        print("loaded_vars",loaded_vars)
        
##        The following for extract global step does not work, it always reset to 0 
#         increment_global_step = graph.get_tensor_by_name("increment_global_step:0").eval()
#         other_var = graph.get_tensor_by_name("enc_fc1/weights:0").eval()
#         print ("Recover global step",increment_global_step)
#         print ("other var",other_var)
    else:
        print("Initializer from scratch")
        global_step = model.sess.run(model.global_step)
    # Training loop    
    for epoch in range(num_epoch):
        model.sess.run(model.train_iterator.initializer)
        model.sess.run(model.val_iterator.initializer)
        start_time = time.time()
        # Run an epoch
        for iter in range(num_sample // batch_size):
            # Get a batch
            #step = epoch*(num_sample // batch_size) +  iter
            print ("global step",model.global_step)
            train_losses,val_losses = model.run_single_step()
            print ("Train_loss: {}; Val_loss{} for global step {}".format(train_losses,val_losses,global_step))
            checkpoint_path = os.path.join(model.checkpoint_dir, 'model_arc4.ckpt')
            model.saver.save(model.sess,checkpoint_path, global_step = global_step)
            global_step = global_step  +1
        end_time = time.time()
        
        # Log the loss
#         if epoch % log_step == 0:
#             log_str = '[Epoch {}] '.format(epoch)
#             for k, v in self.recon_lvoss.items():
#                 log_str += '{}: {:.3f}  '.format(k, v)
#             log_str += '({:.3f} sec/epoch)'.format(end_time - start_time)
#             print(log_str)
    print('Done!')
    return model    
    

In [ ]:
model_vae2 = trainer_and_checkpoint(convLSTM)

Inputs shape (?, 64, 64, 3)
conv_layer activation function relu
DEBUG input shape (?, 64, 64, 3)
Encode_1_shape (?, 32, 32, 8)
conv_layer activation function relu
DEBUG input shape (?, 32, 32, 8)
Encode 2_shape, (?, 32, 32, 8)
conv_layer activation function relu
DEBUG input shape (?, 32, 32, 8)
Encode 3_shape,  (?, 16, 16, 8)
hidden zero layer (?, 16, 16, 16)
y1_shape, [None, 16, 16, 8]
input_channel 8
output_shape Tensor("network/decode_5_trans_conv/stack:0", shape=(4,), dtype=int32)
conv5 shape Tensor("network/decode_5_trans_conv/decode_5_transpose_conv:0", shape=(?, 32, 32, 8), dtype=float32)
input_channel 8
output_shape Tensor("network/decode_6_trans_conv/stack:0", shape=(4,), dtype=int32)
conv6 shape Tensor("network/decode_6_trans_conv/decode_6_transpose_conv:0", shape=(?, 32, 32, 8), dtype=float32)
input_channel 8
output_shape Tensor("network/decode_7_trans_conv/stack:0", shape=(4,), dtype=int32)
x hat shape Tensor("network/decode_7_trans_conv/decode_7_transpose_conv:0", shape=(?

x hat shape Tensor("network_9/decode_7_trans_conv/decode_7_transpose_conv:0", shape=(?, 64, 64, 3), dtype=float32)
Inputs shape (?, 64, 64, 3)
conv_layer activation function relu
DEBUG input shape (?, 64, 64, 3)
Encode_1_shape (?, 32, 32, 8)
conv_layer activation function relu
DEBUG input shape (?, 32, 32, 8)
Encode 2_shape, (?, 32, 32, 8)
conv_layer activation function relu
DEBUG input shape (?, 32, 32, 8)
Encode 3_shape,  (?, 16, 16, 8)
y1_shape, [None, 16, 16, 8]
input_channel 8
output_shape Tensor("network_10/decode_5_trans_conv/stack:0", shape=(4,), dtype=int32)
conv5 shape Tensor("network_10/decode_5_trans_conv/decode_5_transpose_conv:0", shape=(?, 32, 32, 8), dtype=float32)
input_channel 8
output_shape Tensor("network_10/decode_6_trans_conv/stack:0", shape=(4,), dtype=int32)
conv6 shape Tensor("network_10/decode_6_trans_conv/decode_6_transpose_conv:0", shape=(?, 32, 32, 8), dtype=float32)
input_channel 8
output_shape Tensor("network_10/decode_7_trans_conv/stack:0", shape=(4,), d

Encode 3_shape,  (?, 16, 16, 8)
y1_shape, [None, 16, 16, 8]
input_channel 8
output_shape Tensor("network_18/decode_5_trans_conv/stack:0", shape=(4,), dtype=int32)
conv5 shape Tensor("network_18/decode_5_trans_conv/decode_5_transpose_conv:0", shape=(?, 32, 32, 8), dtype=float32)
input_channel 8
output_shape Tensor("network_18/decode_6_trans_conv/stack:0", shape=(4,), dtype=int32)
conv6 shape Tensor("network_18/decode_6_trans_conv/decode_6_transpose_conv:0", shape=(?, 32, 32, 8), dtype=float32)
input_channel 8
output_shape Tensor("network_18/decode_7_trans_conv/stack:0", shape=(4,), dtype=int32)
x hat shape Tensor("network_18/decode_7_trans_conv/decode_7_transpose_conv:0", shape=(?, 64, 64, 3), dtype=float32)
Inputs shape (?, 64, 64, 3)
conv_layer activation function relu
DEBUG input shape (?, 64, 64, 3)
Encode_1_shape (?, 32, 32, 8)
conv_layer activation function relu
DEBUG input shape (?, 32, 32, 8)
Encode 2_shape, (?, 32, 32, 8)
conv_layer activation function relu
DEBUG input shape (?